# Setup and Helper Methods

In [1]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, ShuffleSplit
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss, accuracy_score, precision_recall_curve, average_precision_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler, label_binarize

In [2]:
from wisdm import wisdm
import random
import numpy as np
import pandas as pd
from collections import Counter
import time
from scipy import stats
import os
import json
import plotly.figure_factory as ff

In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [4]:
n_trees = 10000
n_cores = 30
oob_score = False

In [5]:
def shuffle_rows(features, labels):
    permutation = np.random.permutation(features.shape[0])
    return features[permutation], labels[permutation]

In [6]:
def impersonal_pred(model, scaler, test_features):
    scaled_test_features = scaler.transform(test_features)
    
    impersonal_predictions = model.predict(scaled_test_features)
    impersonal_probabilities = model.predict_proba(scaled_test_features)
    return impersonal_predictions, impersonal_probabilities

In [7]:
def personal_pred(personal_features, personal_labels, test_features):
    # build personal model and predict
    personal_scaler = StandardScaler().fit(personal_features)
    scaled_personal_features = personal_scaler.transform(personal_features)
    scaled_test_features = personal_scaler.transform(test_features)

    personal_clf = wisdm.weka_RF()
    personal_clf.set_params(n_estimators=n_trees, n_jobs=n_cores)
    personal_clf.fit(scaled_personal_features, personal_labels)
    
    personal_predictions = personal_clf.predict(scaled_test_features)
    personal_probabilities = personal_clf.predict_proba(scaled_test_features)
    return personal_predictions, personal_probabilities

In [8]:
def filter_dataset(features, labels, labels_to_keep):
    keep_ind = [ind for ind, l in enumerate(labels) if l in labels_to_keep]
    features = features[keep_ind]
    labels = labels[keep_ind]
    
    return features, labels

In [9]:
def hybrid_pred(impersonal_features, impersonal_labels, \
                personal_features, personal_labels, \
                test_features, number_of_samples = None, \
                probabilities=None, \
                sampling_function=None, filter_impersonal=False):
    
    classes_labeled = list(set(personal_labels))
    if filter_impersonal:
        impersonal_features, impersonal_labels = filter_dataset(impersonal_features, impersonal_labels, classes_labeled)
    
    if sampling_function == None:
        hybrid_labels = np.hstack((personal_labels, impersonal_labels))
        hybrid_features = np.vstack((personal_features, impersonal_features))
        hybrid_features, hybrid_labels = shuffle_rows(hybrid_features, hybrid_labels)
    else:
        hybrid_features, hybrid_labels = sampling_function(personal_features, personal_labels, probabilities, number=number_of_samples)
    
    hybrid_scaler = StandardScaler().fit(hybrid_features)
    scaled_hybrid_features = hybrid_scaler.transform(hybrid_features)
    scaled_test_features = hybrid_scaler.transform(test_features)

    hybrid_clf = wisdm.weka_RF()
    hybrid_clf.set_params(n_estimators=n_trees, n_jobs=n_cores)
    hybrid_clf.fit(scaled_hybrid_features, hybrid_labels)
    
    hybrid_predictions = hybrid_clf.predict(scaled_test_features)
    hybrid_probabilities = hybrid_clf.predict_proba(scaled_test_features)
    
    return hybrid_predictions, hybrid_probabilities

In [10]:
def confidence_sample(features, labels, probabilities, number=None, top=False):
    confidence_ranking = np.argsort(np.max(probabilities, axis=1))
    
    if not number:
        return features[confidence_ranking], labels[confidence_ranking]
    
    if top:
        return features[confidence_ranking[-number:]], labels[confidence_ranking[-number:]]
    return features[confidence_ranking[:number]], labels[confidence_ranking[:number]]

# Experiment 1
Running impersonal, personal, and Hybrid models with varying amounts of samples to understand 
accuracy with regard to the UX at different stages of personalization

In [12]:
wisdm.set_data(version="2")

In [13]:
users_to_ignore = []
for user_id in wisdm.user_ids:
    user_df = wisdm.remove_all_nan(wisdm.get_user_set(user_id))
    num_features = len(user_df)
    num_unique_labels = len(user_df['class'].unique())
    
    if num_features < 20:
        users_to_ignore.append(user_id)
    elif num_unique_labels < 2:
        users_to_ignore.append(user_id)


In [14]:
def experiment1(training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels):
    result_rows = []

    ignored_users = []
    print("predicting...")
    start_predicting = time.time()
    for user_id in wisdm.user_ids:
        print("User : %s" % user_id)
        user_df = wisdm.data_df[wisdm.data_df['user'] == user_id]

        if (len(user_df) < (training_size + test_size)) or (user_id in users_to_ignore):
            print("Not Enough Data, skipping...")
            ignored_users.append(user_id)
            continue

        personal_labels = np.array([t.decode("utf-8") for t in user_df['class'].as_matrix()])
        personal_features = user_df.as_matrix(columns=[user_df.columns[1:-1]])

        sss = StratifiedShuffleSplit(n_splits=4, test_size=test_size, train_size = training_size)

        personal_scores = []
        impersonal_scores = []
        hybrid_scores = []

        shuffle_count = 0
        try:
            for train_index, test_index in sss.split(personal_features, personal_labels):
                # data for personal model
                random_personal_features = personal_features[train_index]
                random_personal_labels = personal_labels[train_index]

                # create an active pool of everything not in the test set for active learning / hybrid model
                #active_pool_mask = np.ones(personal_labels.shape, dtype=bool)
                #active_pool_mask[test_index] = False
                #active_pool_features = personal_features[active_pool_mask]
                #active_pool_labels = personal_labels[active_pool_mask]

                # test set
                test_features = personal_features[test_index]
                test_labels = personal_labels[test_index]
                
                personal_counts = Counter(random_personal_labels)
                test_counts = Counter(test_labels)
                if shuffle_count == 0:
                    print("\tPersonal Training Counts : %s" % personal_counts)
                    print("\tPersonal Test Counts : %s" % test_counts)

                # build personal model and predict
                personal_predictions, _ = personal_pred(random_personal_features, random_personal_labels, test_features)
                personal_score = accuracy_score(test_labels, personal_predictions)
                personal_scores.append(personal_score)

                # build impersonal model and predict
                impersonal_predictions, impersonal_probabilities = impersonal_pred(impersonal_clf, impersonal_scaler, test_features)
                impersonal_score = accuracy_score(test_labels, impersonal_predictions)
                impersonal_scores.append(impersonal_score)

                # build hybrid model and predict
                hybrid_predictions, _ = hybrid_pred(impersonal_features, impersonal_labels, \
                                                    random_personal_features, random_personal_labels, \
                                                    test_features)

                hybrid_score = accuracy_score(test_labels, hybrid_predictions)
                hybrid_scores.append(hybrid_score)
                #print("\t impersonal acc : %.3f" % impersonal_score)
                #print("\t personal acc : %.3f" % personal_score)
                #print("\t hybrid acc : %.3f" % hybrid_score)
                #print("")

                result_row = {"user_id" : user_id,
                              "shuffle" : shuffle_count,
                              "impersonal" : impersonal_score,
                              "personal" : personal_score,
                              "hybrid" : hybrid_score,
                              "training size" : training_size,
                              "test size" : test_size}
                result_rows.append(result_row)
                shuffle_count += 1
        except ValueError as ve:
            if "The least populated class" in ve.args[0]:
                print("\tNot enough labeled data for %s" % user_id)
                ignored_users.append(user_id)
                continue
            else:
                raise ve
        print("")
        print("\t impersonal acc : %.3f" % np.mean(impersonal_scores))
        print("\t personal acc : %.3f" % np.mean(personal_score))
        print("\t hybrid acc : %.3f" % np.mean(hybrid_score))
        print("")

    finished_predicting = time.time()
    print("Finished predicting in %s seconds" % (finished_predicting - start_predicting))
    print("Users without enough data : %s" % ignored_users)

    results_df = pd.DataFrame(result_rows)
    return results_df

In [ ]:
training_sizes = [5,8,10,15,20]
test_size = 30
output_path = "./results/experiment1_10-9-17/"
results = []

# set impersonal model details
wisdm.set_data(version="1", make_compatible=True)
impersonal_df = wisdm.remove_all_nan(wisdm.data_df)
impersonal_labels = np.array([t.decode("utf-8") for t in impersonal_df['class'].as_matrix()])
impersonal_features = impersonal_df.as_matrix(columns=[impersonal_df.columns[1:-1]])
impersonal_scaler = StandardScaler().fit(impersonal_features)
scaled_train_X = impersonal_scaler.transform(impersonal_features)
impersonal_clf = wisdm.weka_RF()
impersonal_clf.set_params(n_estimators=n_trees, n_jobs=n_cores)

start=time.time()
print("Training Impersonal Model...")
impersonal_clf.fit(scaled_train_X, impersonal_labels)
finished_training = time.time()
print("Finished Training in %s seconds" % (finished_training - start))

wisdm.set_data(version="2", make_compatible=True)
for training_size in training_sizes:
    print("Personal Training Size : %s" % training_size)
    results_df = experiment1(training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels)
    results_df.to_pickle(output_path + "training_size-"+str(training_size)+".pickle")
    results.append(results_df)
    
results_df = pd.concat(results)

Training Impersonal Model...
Finished Training in 23.857114553451538 seconds
Personal Training Size : 5
predicting...
User : 194
	Personal Training Counts : Counter({'Standing': 2, 'Jogging': 1, 'Stairs': 1, 'Walking': 1})
	Personal Test Counts : Counter({'Standing': 12, 'Stairs': 7, 'Walking': 6, 'Jogging': 3, 'Sitting': 2})

	 impersonal acc : 0.717
	 personal acc : 0.867
	 hybrid acc : 0.867

User : 998
Not Enough Data, skipping...
User : 1097
Not Enough Data, skipping...
User : 1104
	Personal Training Counts : Counter({'Walking': 2, 'Jogging': 1, 'Stairs': 1, 'Standing': 1})
	Personal Test Counts : Counter({'Walking': 12, 'Standing': 8, 'Jogging': 4, 'Stairs': 3, 'Sitting': 3})

	 impersonal acc : 0.525
	 personal acc : 0.500
	 hybrid acc : 0.833

User : 1117
	Personal Training Counts : Counter({'Standing': 3, 'Stairs': 1, 'Sitting': 1})
	Personal Test Counts : Counter({'Standing': 18, 'Sitting': 8, 'Stairs': 4})

	 impersonal acc : 0.650
	 personal acc : 0.833
	 hybrid acc : 0.867

# This is just to correct Experiment1

In [93]:
training_sizes = [8]
test_size = 30
output_path1 = "./results/experiment1_10-4-17/"
results = []

# set impersonal model details
wisdm.set_data(version="1", make_compatible=True)
impersonal_df = wisdm.remove_all_nan(wisdm.data_df)
impersonal_labels = np.array([t.decode("utf-8") for t in impersonal_df['class'].as_matrix()])
impersonal_features = impersonal_df.as_matrix(columns=[impersonal_df.columns[1:-1]])
impersonal_scaler = StandardScaler().fit(impersonal_features)
scaled_train_X = impersonal_scaler.transform(impersonal_features)
impersonal_clf = wisdm.weka_RF()
impersonal_clf.set_params(n_estimators=n_trees, n_jobs=n_cores)

start=time.time()
print("Training Impersonal Model...")
impersonal_clf.fit(scaled_train_X, impersonal_labels)
finished_training = time.time()
print("Finished Training in %s seconds" % (finished_training - start))

wisdm.set_data(version="2", make_compatible=True)
for training_size in training_sizes:
    print("Personal Training Size : %s" % training_size)
    results_df = experiment1(training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels)
    results_df.to_pickle(output_path + "training_size-"+str(training_size)+".pickle")
    results.append(results_df)
    
results_df = pd.concat(results)

Training Impersonal Model...
Finished Training in 23.07707667350769 seconds
Personal Training Size : 8
predicting...
User : 194
	Personal Training Counts : Counter({'Standing': 3, 'Stairs': 2, 'Walking': 1, 'Sitting': 1, 'Jogging': 1})
	Personal Test Counts : Counter({'Standing': 12, 'Stairs': 7, 'Walking': 6, 'Jogging': 3, 'Sitting': 2})

	 impersonal acc : 0.733
	 personal acc : 0.933
	 hybrid acc : 0.900

User : 998
	Personal Training Counts : Counter({'Walking': 8})
	Personal Test Counts : Counter({'Walking': 30})

	 impersonal acc : 0.000
	 personal acc : 1.000
	 hybrid acc : 1.000

User : 1097
Not Enough Data, skipping...
User : 1104
	Personal Training Counts : Counter({'Walking': 3, 'Standing': 2, 'Sitting': 1, 'Jogging': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Walking': 12, 'Standing': 8, 'Jogging': 4, 'Sitting': 3, 'Stairs': 3})

	 impersonal acc : 0.558
	 personal acc : 0.833
	 hybrid acc : 0.767

User : 1117
	Personal Training Counts : Counter({'Standing': 5, 'Sit

In [19]:
results_df.head()

,hybrid,impersonal,personal,shuffle,test size,training size,user_id
0,0.900000,0.666667,0.900000,0,30,5,194
1,0.833333,0.766667,0.933333,1,30,5,194
2,0.900000,0.700000,0.933333,2,30,5,194
3,0.900000,0.666667,0.900000,3,30,5,194
4,1.000000,0.000000,1.000000,0,30,5,998


In [97]:
output_path1 = "./results/experiment1_10-4-17/"

In [99]:
pickle_files = [output_path1 + f for f in os.listdir(output_path1) if ".pickle" in f]
results = [pd.read_pickle(f) for f in pickle_files]
results_df1 = pd.concat(results)

In [105]:
results_df1['training size'].unique()

array([ 5, 10, 15, 20,  8])

In [106]:
impersonal_means = []
personal_means = []
hybrid_means = []

training_sizes = results_df1['training size'].unique()

for training_size in training_sizes:
    for user_id in wisdm.user_ids:
        user_df = results_df1[(results_df1['user_id'] == user_id) & \
                             (results_df1['training size'] == training_size)]
        if training_size == 8:
            impersonal_means.append((0, user_df['impersonal'].mean()))
        personal_means.append((training_size, user_df['personal'].mean()))
        hybrid_means.append((training_size, user_df['hybrid'].mean()))
        
impersonal_x, impersonal_y = zip(*impersonal_means)
impersonal_trace = go.Box(y=impersonal_y,
                          x=impersonal_x,
                          name="Impersonal",
                          marker=dict(color="red"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

personal_x, personal_y = zip(*personal_means)
personal_trace = go.Box(y=personal_y,
                          x=personal_x,
                          name="Personal",
                          marker=dict(color="blue"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

hybrid_x, hybrid_y = zip(*hybrid_means)
hybrid_trace = go.Box(y=hybrid_y,
                          x=hybrid_x,
                          name="Hybrid",
                          marker=dict(color="green"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

layout=go.Layout(boxmode="group", xaxis=dict(title="number of personal training samples"), yaxis=dict(title="accuracy"))

data = [impersonal_trace, personal_trace, hybrid_trace]

fig = go.Figure(data=data, layout=layout)

iplot(fig)

# Experiment 1A
What is the role that class balance plays here?

# TODO : plot accuracy with colored dots where color is equal to the number of classes the user has labeled.

In [12]:
# number of labels by participant
user_num_labels = {user_id : len(wisdm.data_df[wisdm.data_df['user'] == user_id]['class'].unique()) for user_id in wisdm.user_ids}

In [125]:
# distribution of class labels for participants
import plotly.figure_factory as ff

hist_data = [l for l in user_num_labels.values() if l > 0]
group_labels = ['number of users']
#fig = ff.create_distplot(hist_data, group_labels, show_rug=False, show_curve=False, histnorm='probability density')
data = [go.Histogram(x=hist_data)]
layout = go.Layout(yaxis=dict(title="Number Of Users"),
                   xaxis=dict(title="Number Of Classes Labeled"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [126]:
users_with_1_class = [user_id for user_id, l in user_num_labels.items() if l == 1]

In [129]:
users_with_4_class = [user_id for user_id, l in user_num_labels.items() if l == 4]

In [131]:
impersonal_means = []
personal_means = []
hybrid_means = []

training_sizes = results_df1['training size'].unique()

for training_size in training_sizes:
    for user_id in wisdm.user_ids:
        if user_id in users_with_1_class:
            continue
        user_df = results_df1[(results_df1['user_id'] == user_id) & \
                             (results_df1['training size'] == training_size)]
        if training_size == 8:
            impersonal_means.append((0, user_df['impersonal'].mean()))
        personal_means.append((training_size, user_df['personal'].mean()))
        hybrid_means.append((training_size, user_df['hybrid'].mean()))
        
impersonal_x, impersonal_y = zip(*impersonal_means)
impersonal_trace = go.Box(y=impersonal_y,
                          x=impersonal_x,
                          name="Impersonal",
                          marker=dict(color="red"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

personal_x, personal_y = zip(*personal_means)
personal_trace = go.Box(y=personal_y,
                          x=personal_x,
                          name="Personal",
                          marker=dict(color="blue"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

hybrid_x, hybrid_y = zip(*hybrid_means)
hybrid_trace = go.Box(y=hybrid_y,
                          x=hybrid_x,
                          name="Hybrid",
                          marker=dict(color="green"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

layout=go.Layout(boxmode="group", xaxis=dict(title="number of personal training samples"), yaxis=dict(title="accuracy"))

data = [impersonal_trace, personal_trace, hybrid_trace]

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [134]:
def experiment1A(training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels):
    result_rows = []

    ignored_users = []
    print("predicting...")
    start_predicting = time.time()
    for user_id in wisdm.user_ids:
        if user_id in users_to_ignore:
            ignored_users.append(user_id)
            continue
        
        print("User : %s" % user_id)

        user_df = wisdm.data_df[wisdm.data_df['user'] == user_id]

        if len(user_df) < (training_size + test_size):
            print("Not Enough Data, skipping...")
            ignored_users.append(user_id)
            continue

        personal_labels = np.array([t.decode("utf-8") for t in user_df['class'].as_matrix()])
        personal_features = user_df.as_matrix(columns=[user_df.columns[1:-1]])

        sss = StratifiedShuffleSplit(n_splits=4, test_size=test_size, train_size = training_size)

        personal_scores = []
        impersonal_scores = []
        hybrid_scores = []

        shuffle_count = 0
        try:
            for train_index, test_index in sss.split(personal_features, personal_labels):
                # data for personal model
                random_personal_features = personal_features[train_index]
                random_personal_labels = personal_labels[train_index]

                # create an active pool of everything not in the test set for active learning / hybrid model
                #active_pool_mask = np.ones(personal_labels.shape, dtype=bool)
                #active_pool_mask[test_index] = False
                #active_pool_features = personal_features[active_pool_mask]
                #active_pool_labels = personal_labels[active_pool_mask]

                # test set
                test_features = personal_features[test_index]
                test_labels = personal_labels[test_index]
                
                personal_counts = Counter(random_personal_labels)
                test_counts = Counter(test_labels)
                if shuffle_count == 0:
                    print("\tPersonal Training Counts : %s" % personal_counts)
                    print("\tPersonal Test Counts : %s" % test_counts)

                # build personal model and predict
                personal_predictions, _ = personal_pred(random_personal_features, random_personal_labels, test_features)
                personal_score = accuracy_score(test_labels, personal_predictions)
                personal_scores.append(personal_score)

                # build impersonal model and predict
                impersonal_predictions, impersonal_probabilities = impersonal_pred(impersonal_clf, impersonal_scaler, test_features)
                impersonal_score = accuracy_score(test_labels, impersonal_predictions)
                impersonal_scores.append(impersonal_score)

                # build hybrid model and predict
                hybrid_predictions, _ = hybrid_pred(impersonal_features, impersonal_labels, \
                                                    random_personal_features, random_personal_labels, \
                                                    test_features, filter_impersonal=True)

                hybrid_score = accuracy_score(test_labels, hybrid_predictions)
                hybrid_scores.append(hybrid_score)
                #print("\t impersonal acc : %.3f" % impersonal_score)
                #print("\t personal acc : %.3f" % personal_score)
                #print("\t hybrid acc : %.3f" % hybrid_score)
                #print("")

                result_row = {"user_id" : user_id,
                              "shuffle" : shuffle_count,
                              "impersonal" : impersonal_score,
                              "personal" : personal_score,
                              "hybrid" : hybrid_score,
                              "training size" : training_size,
                              "test size" : test_size}
                result_rows.append(result_row)
                shuffle_count += 1
        except ValueError as ve:
            if "The least populated class" in ve.args[0]:
                print("\tNot enough labeled data for %s" % user_id)
                ignored_users.append(user_id)
                continue
            else:
                raise ve
        print("")
        print("\t impersonal acc : %.3f" % np.mean(impersonal_scores))
        print("\t personal acc : %.3f" % np.mean(personal_score))
        print("\t hybrid acc : %.3f" % np.mean(hybrid_score))
        print("")

    finished_predicting = time.time()
    print("Finished predicting in %s seconds" % (finished_predicting - start_predicting))
    print("Users without enough data : %s" % ignored_users)

    results_df = pd.DataFrame(result_rows)
    return results_df

In [141]:
training_sizes = [5,8,10,15,20]
test_size = 30
output_path1 = "./results/experiment1a_10-6-17"
results = []

# set impersonal model details
wisdm.set_data(version="1", make_compatible=True)
impersonal_df = wisdm.remove_all_nan(wisdm.data_df)
impersonal_labels = np.array([t.decode("utf-8") for t in impersonal_df['class'].as_matrix()])
impersonal_features = impersonal_df.as_matrix(columns=[impersonal_df.columns[1:-1]])
impersonal_scaler = StandardScaler().fit(impersonal_features)
scaled_train_X = impersonal_scaler.transform(impersonal_features)
impersonal_clf = wisdm.weka_RF()
impersonal_clf.set_params(n_estimators=n_trees, n_jobs=n_cores)

start=time.time()
print("Training Impersonal Model...")
impersonal_clf.fit(scaled_train_X, impersonal_labels)
finished_training = time.time()
print("Finished Training in %s seconds" % (finished_training - start))

wisdm.set_data(version="2", make_compatible=True)
for training_size in training_sizes:
    print("Personal Training Size : %s" % training_size)
    results_df = experiment1A(training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels)
    results_df.to_pickle(output_path1 + "training_size-"+str(training_size)+".pickle")
    results.append(results_df)
    
results_df = pd.concat(results)

Training Impersonal Model...
Finished Training in 23.2321834564209 seconds
Personal Training Size : 5
predicting...
User : 194
	Personal Training Counts : Counter({'Standing': 2, 'Walking': 1, 'Jogging': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Standing': 12, 'Stairs': 7, 'Walking': 6, 'Jogging': 3, 'Sitting': 2})

	 impersonal acc : 0.725
	 personal acc : 0.900
	 hybrid acc : 0.900

User : 1104
	Personal Training Counts : Counter({'Walking': 2, 'Standing': 1, 'Jogging': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Walking': 12, 'Standing': 8, 'Jogging': 4, 'Sitting': 3, 'Stairs': 3})

	 impersonal acc : 0.592
	 personal acc : 0.733
	 hybrid acc : 0.767

User : 1117
	Personal Training Counts : Counter({'Standing': 3, 'Sitting': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Standing': 18, 'Sitting': 8, 'Stairs': 4})

	 impersonal acc : 0.658
	 personal acc : 0.767
	 hybrid acc : 0.733

User : 1238
	Personal Training Counts : Counter({'Standing': 3, 'Walking': 2})
	P


	 impersonal acc : 0.125
	 personal acc : 0.833
	 hybrid acc : 0.833

User : 1703
	Personal Training Counts : Counter({'Standing': 4, 'Sitting': 1, 'Walking': 1, 'Jogging': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Standing': 16, 'Sitting': 5, 'Walking': 5, 'Jogging': 2, 'Stairs': 2})

	 impersonal acc : 0.475
	 personal acc : 0.933
	 hybrid acc : 0.967

User : 1707
	Personal Training Counts : Counter({'Standing': 4, 'Walking': 2, 'Sitting': 1, 'Jogging': 1})
	Personal Test Counts : Counter({'Standing': 13, 'Walking': 9, 'Sitting': 4, 'Jogging': 4})

	 impersonal acc : 0.267
	 personal acc : 0.600
	 hybrid acc : 0.533

User : 1742
	Personal Training Counts : Counter({'Walking': 3, 'Sitting': 2, 'Stairs': 2, 'Standing': 1})
	Personal Test Counts : Counter({'Walking': 12, 'Stairs': 7, 'Standing': 6, 'Sitting': 5})

	 impersonal acc : 0.600
	 personal acc : 1.000
	 hybrid acc : 0.967

User : 1758
	Not enough labeled data for 1758
User : 1759
	Personal Training Counts : Counter({


	 impersonal acc : 0.417
	 personal acc : 0.733
	 hybrid acc : 0.767

User : 1809
	Personal Training Counts : Counter({'Sitting': 5, 'Walking': 4, 'Standing': 1})
	Personal Test Counts : Counter({'Sitting': 14, 'Walking': 12, 'Standing': 4})

	 impersonal acc : 0.050
	 personal acc : 0.933
	 hybrid acc : 0.933

Finished predicting in 3945.5736281871796 seconds
Users without enough data : ['998', '1097', '1205', '1246', '1247', '1269', '1274', '1276', '1277', '1280', '1480', '1491', '1511', '1518', '1531', '1554', '1559', '1679', '1683', '1696', '1723', '1724', '1726', '1745', '1750', '1757', '1758', '1761', '1763', '1797', '1802', '1813', '1814']
Personal Training Size : 15
predicting...
User : 194
	Personal Training Counts : Counter({'Standing': 6, 'Walking': 3, 'Stairs': 3, 'Jogging': 2, 'Sitting': 1})
	Personal Test Counts : Counter({'Standing': 12, 'Stairs': 7, 'Walking': 6, 'Jogging': 3, 'Sitting': 2})

	 impersonal acc : 0.758
	 personal acc : 0.967
	 hybrid acc : 0.933

User : 


	 impersonal acc : 0.142
	 personal acc : 0.967
	 hybrid acc : 0.967

User : 1477
	Personal Training Counts : Counter({'Walking': 19, 'Standing': 1})
	Personal Test Counts : Counter({'Walking': 28, 'Standing': 2})

	 impersonal acc : 0.033
	 personal acc : 1.000
	 hybrid acc : 1.000

User : 1512
	Personal Training Counts : Counter({'Walking': 18, 'Sitting': 1, 'Jogging': 1})
	Personal Test Counts : Counter({'Walking': 28, 'Sitting': 1, 'Jogging': 1})

	 impersonal acc : 0.067
	 personal acc : 0.967
	 hybrid acc : 1.000

User : 1603
	Personal Training Counts : Counter({'Sitting': 10, 'Standing': 8, 'Walking': 2})
	Personal Test Counts : Counter({'Sitting': 14, 'Standing': 12, 'Walking': 4})

	 impersonal acc : 0.783
	 personal acc : 1.000
	 hybrid acc : 1.000

User : 1676
	Personal Training Counts : Counter({'Jogging': 7, 'Walking': 6, 'Standing': 5, 'Sitting': 2})
	Personal Test Counts : Counter({'Walking': 9, 'Jogging': 9, 'Standing': 8, 'Sitting': 4})

	 impersonal acc : 0.100
	 per

In [142]:
results_df1a = results_df

In [143]:
impersonal_means = []
personal_means = []
hybrid_means = []

training_sizes = results_df1a['training size'].unique()

for training_size in training_sizes:
    for user_id in wisdm.user_ids:
        if user_id in users_with_1_class:
            continue
        user_df = results_df1a[(results_df1a['user_id'] == user_id) & \
                             (results_df1a['training size'] == training_size)]
        if training_size == 8:
            impersonal_means.append((0, user_df['impersonal'].mean()))
        personal_means.append((training_size, user_df['personal'].mean()))
        hybrid_means.append((training_size, user_df['hybrid'].mean()))
        
impersonal_x, impersonal_y = zip(*impersonal_means)
impersonal_trace = go.Box(y=impersonal_y,
                          x=impersonal_x,
                          name="Impersonal",
                          marker=dict(color="red"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

personal_x, personal_y = zip(*personal_means)
personal_trace = go.Box(y=personal_y,
                          x=personal_x,
                          name="Personal",
                          marker=dict(color="blue"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

hybrid_x, hybrid_y = zip(*hybrid_means)
hybrid_trace = go.Box(y=hybrid_y,
                          x=hybrid_x,
                          name="Hybrid",
                          marker=dict(color="green"),
                          boxpoints='all',
                          jitter=0.3,
                          pointpos=-1)

layout=go.Layout(boxmode="group", xaxis=dict(title="number of personal training samples"), yaxis=dict(title="accuracy"))

data = [impersonal_trace, personal_trace, hybrid_trace]

fig = go.Figure(data=data, layout=layout)

iplot(fig)

# Experiment 2

In [ ]:
def experiment2(output_file_loc, training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels):
    result_rows = []

    ignored_users = []
    print("predicting...")
    start_predicting = time.time()
    for user_id in wisdm.user_ids:
        if user_id in users_to_ignore:
            ignored_users.append(user_id)
            continue
        
        print("User : %s" % user_id)

        user_df = wisdm.data_df[wisdm.data_df['user'] == user_id]

        if len(user_df) < (training_size + test_size):
            print("Not Enough Data, skipping...")
            ignored_users.append(user_id)
            continue

        personal_labels = np.array([t.decode("utf-8") for t in user_df['class'].as_matrix()])
        personal_features = user_df.as_matrix(columns=[user_df.columns[1:-1]])

        sss = StratifiedShuffleSplit(n_splits=4, test_size=test_size, train_size = training_size)

        personal_scores = []
        impersonal_scores = []
        hybrid_scores = []

        shuffle_count = 0
        try:
            for train_index, test_index in sss.split(personal_features, personal_labels):
                # data for personal model
                random_personal_features = personal_features[train_index]
                random_personal_labels = personal_labels[train_index]

                # create an active pool of everything not in the test set for active learning / hybrid model
                #active_pool_mask = np.ones(personal_labels.shape, dtype=bool)
                #active_pool_mask[test_index] = False
                #active_pool_features = personal_features[active_pool_mask]
                #active_pool_labels = personal_labels[active_pool_mask]

                # test set
                test_features = personal_features[test_index]
                test_labels = personal_labels[test_index]
                
                personal_counts = Counter(random_personal_labels)
                test_counts = Counter(test_labels)
                if shuffle_count == 0:
                    print("\tPersonal Training Counts : %s" % personal_counts)
                    print("\tPersonal Test Counts : %s" % test_counts)

                # build personal model and predict
                personal_predictions, personal_probabilities = personal_pred(random_personal_features, random_personal_labels, test_features)
                personal_score = accuracy_score(test_labels, personal_predictions)
                personal_scores.append(personal_score)

                # build impersonal model and predict
                impersonal_predictions, impersonal_probabilities = impersonal_pred(impersonal_clf, impersonal_scaler, test_features)
                impersonal_score = accuracy_score(test_labels, impersonal_predictions)
                impersonal_scores.append(impersonal_score)

                # build hybrid model and predict
                hybrid_predictions, hybrid_probabilities = hybrid_pred(impersonal_features, impersonal_labels, \
                                                    random_personal_features, random_personal_labels, \
                                                    test_features)

                hybrid_score = accuracy_score(test_labels, hybrid_predictions)
                hybrid_scores.append(hybrid_score)
                #print("\t impersonal acc : %.3f" % impersonal_score)
                #print("\t personal acc : %.3f" % personal_score)
                #print("\t hybrid acc : %.3f" % hybrid_score)
                #print("")

                result_row = {"user_id" : user_id,
                      "training size" : training_size,
                      "impersonal probabilities" : [list(ip) for ip in impersonal_probabilities],
                      "impersonal predictions" : list(impersonal_predictions),
                      "personal probabilities" : [list(pp) for pp in personal_probabilities],
                      "personal predictions" : list(personal_predictions),
                      "hybrid probabilities" : [list(hp) for hp in hybrid_probabilities],
                      "hybrid predictions" : list(hybrid_predictions),
                      "true" : list(personal_labels)}
                #result_rows.append(result_row)
                
                with open(output_file_loc, "a") as f_out:
                    try:
                        f_out.write(json.dumps(result_row))
                        f_out.write("\n")
                    except TypeError as te:
                        print(result_row)
                        raise te
                
                shuffle_count += 1
        except ValueError as ve:
            if "The least populated class" in ve.args[0]:
                print("\tNot enough labeled data for %s" % user_id)
                ignored_users.append(user_id)
                continue
            else:
                raise ve
        print("")
        print("\t impersonal acc : %.3f" % np.mean(impersonal_scores))
        print("\t personal acc : %.3f" % np.mean(personal_score))
        print("\t hybrid acc : %.3f" % np.mean(hybrid_score))
        print("")

    finished_predicting = time.time()
    print("Finished predicting in %s seconds" % (finished_predicting - start_predicting))
    print("Users without enough data : %s" % ignored_users)

    #results_df = pd.DataFrame(result_rows)
    #return results_df

In [35]:
training_sizes = [5,8,10,15,20]
test_size = 30
output_path2 = "./results/experiments2_10-6-17/"
file_name = "experiment2.json"
output_file_loc = output_path2+file_name
#os.remove(output_file_loc)
results = []

# set impersonal model details
wisdm.set_data(version="1", make_compatible=True)
impersonal_df = wisdm.remove_all_nan(wisdm.data_df)
impersonal_labels = np.array([t.decode("utf-8") for t in impersonal_df['class'].as_matrix()])
impersonal_features = impersonal_df.as_matrix(columns=[impersonal_df.columns[1:-1]])
impersonal_scaler = StandardScaler().fit(impersonal_features)
scaled_train_X = impersonal_scaler.transform(impersonal_features)
impersonal_clf = wisdm.weka_RF()
impersonal_clf.set_params(n_estimators=n_trees, n_jobs=n_cores)

start=time.time()
print("Training Impersonal Model...")
impersonal_clf.fit(scaled_train_X, impersonal_labels)
finished_training = time.time()
print("Finished Training in %s seconds" % (finished_training - start))

wisdm.set_data(version="2", make_compatible=True)
for training_size in training_sizes:
    print("Personal Training Size : %s" % training_size)
    results_df = experiment2(output_file_loc, training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels)
    #results_df.to_pickle(output_path2 + "training_size-"+str(training_size)+".pickle")
    #results.append(results_df)
    
#results_df = pd.concat(results)

Training Impersonal Model...
Finished Training in 23.707794427871704 seconds
Personal Training Size : 5
predicting...
User : 194
	Personal Training Counts : Counter({'Standing': 2, 'Walking': 1, 'Stairs': 1, 'Jogging': 1})
	Personal Test Counts : Counter({'Standing': 12, 'Stairs': 7, 'Walking': 6, 'Jogging': 3, 'Sitting': 2})

	 impersonal acc : 0.708
	 personal acc : 0.900
	 hybrid acc : 0.900

User : 1104
	Personal Training Counts : Counter({'Walking': 2, 'Standing': 1, 'Stairs': 1, 'Jogging': 1})
	Personal Test Counts : Counter({'Walking': 12, 'Standing': 8, 'Jogging': 4, 'Stairs': 3, 'Sitting': 3})

	 impersonal acc : 0.533
	 personal acc : 0.767
	 hybrid acc : 0.800

User : 1117
	Personal Training Counts : Counter({'Standing': 3, 'Sitting': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Standing': 18, 'Sitting': 8, 'Stairs': 4})

	 impersonal acc : 0.650
	 personal acc : 0.800
	 hybrid acc : 0.833

User : 1238
	Personal Training Counts : Counter({'Standing': 3, 'Walking': 2})



	 impersonal acc : 0.017
	 personal acc : 0.800
	 hybrid acc : 0.800

User : 1703
	Personal Training Counts : Counter({'Standing': 4, 'Walking': 1, 'Stairs': 1, 'Jogging': 1, 'Sitting': 1})
	Personal Test Counts : Counter({'Standing': 16, 'Sitting': 5, 'Walking': 5, 'Stairs': 2, 'Jogging': 2})

	 impersonal acc : 0.475
	 personal acc : 0.833
	 hybrid acc : 0.900

User : 1707
	Personal Training Counts : Counter({'Standing': 4, 'Walking': 2, 'Jogging': 1, 'Sitting': 1})
	Personal Test Counts : Counter({'Standing': 13, 'Walking': 9, 'Sitting': 4, 'Jogging': 4})

	 impersonal acc : 0.275
	 personal acc : 0.533
	 hybrid acc : 0.500

User : 1742
	Personal Training Counts : Counter({'Walking': 3, 'Sitting': 2, 'Stairs': 2, 'Standing': 1})
	Personal Test Counts : Counter({'Walking': 12, 'Stairs': 7, 'Sitting': 6, 'Standing': 5})

	 impersonal acc : 0.608
	 personal acc : 0.967
	 hybrid acc : 1.000

User : 1758
	Not enough labeled data for 1758
User : 1759
	Personal Training Counts : Counter({


	 impersonal acc : 0.492
	 personal acc : 0.667
	 hybrid acc : 0.767

User : 1809
	Personal Training Counts : Counter({'Sitting': 5, 'Walking': 4, 'Standing': 1})
	Personal Test Counts : Counter({'Sitting': 14, 'Walking': 12, 'Standing': 4})

	 impersonal acc : 0.050
	 personal acc : 0.967
	 hybrid acc : 0.900

Finished predicting in 4235.367138624191 seconds
Users without enough data : ['998', '1097', '1205', '1246', '1247', '1269', '1274', '1276', '1277', '1280', '1480', '1491', '1511', '1518', '1531', '1554', '1559', '1679', '1683', '1696', '1723', '1724', '1726', '1745', '1750', '1757', '1758', '1761', '1763', '1797', '1802', '1813', '1814']
Personal Training Size : 15
predicting...
User : 194
	Personal Training Counts : Counter({'Standing': 6, 'Walking': 3, 'Stairs': 3, 'Jogging': 2, 'Sitting': 1})
	Personal Test Counts : Counter({'Standing': 12, 'Stairs': 7, 'Walking': 6, 'Jogging': 3, 'Sitting': 2})

	 impersonal acc : 0.708
	 personal acc : 0.967
	 hybrid acc : 1.000

User : 1


	 impersonal acc : 0.192
	 personal acc : 0.933
	 hybrid acc : 0.933

User : 1477
	Personal Training Counts : Counter({'Walking': 19, 'Standing': 1})
	Personal Test Counts : Counter({'Walking': 28, 'Standing': 2})

	 impersonal acc : 0.050
	 personal acc : 1.000
	 hybrid acc : 1.000

User : 1512
	Personal Training Counts : Counter({'Walking': 18, 'Jogging': 1, 'Sitting': 1})
	Personal Test Counts : Counter({'Walking': 28, 'Jogging': 1, 'Sitting': 1})

	 impersonal acc : 0.067
	 personal acc : 0.967
	 hybrid acc : 1.000

User : 1603
	Personal Training Counts : Counter({'Sitting': 10, 'Standing': 8, 'Walking': 2})
	Personal Test Counts : Counter({'Sitting': 14, 'Standing': 12, 'Walking': 4})

	 impersonal acc : 0.792
	 personal acc : 1.000
	 hybrid acc : 0.967

User : 1676
	Personal Training Counts : Counter({'Jogging': 7, 'Walking': 6, 'Standing': 5, 'Sitting': 2})
	Personal Test Counts : Counter({'Walking': 9, 'Jogging': 9, 'Standing': 8, 'Sitting': 4})

	 impersonal acc : 0.017
	 per

In [28]:
json.dumps(list([ 0.0199,  0.055 ,  0.2724,  0.2172,  0.4355]))

'[0.0199, 0.055, 0.2724, 0.2172, 0.4355]'

In [75]:
def experiment2(impersonal_clf, impersonal_scaler):
    result_rows = []

    ignored_users = []
    print("predicting...")
    start_predicting = time.time()
    for user_id in wisdm.user_ids:
        print("User : %s" % user_id)
        user_df = wisdm.data_df[wisdm.data_df['user'] == user_id]

        if len(user_df) < 40:
            print("Not Enough Data, skipping...")
            ignored_users.append(user_id)
            continue

        personal_labels = np.array([t.decode("utf-8") for t in user_df['class'].as_matrix()])
        personal_features = user_df.as_matrix(columns=[user_df.columns[1:-1]])
        
        personal_counts = Counter(personal_labels)
        print("\tPersonal Testing Counts : %s" % personal_counts)

        # build impersonal model and predict
        impersonal_predictions, impersonal_probabilities = impersonal_pred(impersonal_clf, impersonal_scaler, personal_features)


        result_row = {"user_id" : user_id,
                      "probabilities" : impersonal_probabilities,
                      "predictions" : impersonal_predictions,
                      "true" : personal_labels}
        result_rows.append(result_row)

    finished_predicting = time.time()
    print("Finished predicting in %s seconds" % (finished_predicting - start_predicting))
    print("Users without enough data : %s" % ignored_users)

    results_df = pd.DataFrame(result_rows)
    return results_df

In [146]:
output_path = "./results/experiment2_10-5-17/"

# set impersonal model details
wisdm.set_data(version="1", make_compatible=True)
impersonal_df = wisdm.remove_all_nan(wisdm.data_df)
impersonal_labels = np.array([t.decode("utf-8") for t in impersonal_df['class'].as_matrix()])
impersonal_features = impersonal_df.as_matrix(columns=[impersonal_df.columns[1:-1]])
impersonal_scaler = StandardScaler().fit(impersonal_features)
scaled_train_X = impersonal_scaler.transform(impersonal_features)
impersonal_clf = wisdm.weka_RF()
impersonal_clf.set_params(n_estimators=n_trees, n_jobs=n_cores)

start=time.time()
print("Training Impersonal Model...")
impersonal_clf.fit(scaled_train_X, impersonal_labels)
finished_training = time.time()
print("Finished Training in %s seconds" % (finished_training - start))

wisdm.set_data(version="2", make_compatible=True)
print("Making predictions...")
results_df2 = experiment2(impersonal_clf, impersonal_scaler)
results_df2.to_pickle(output_path+"experiment2.pickle")

Training Impersonal Model...
Finished Training in 23.126187324523926 seconds
Making predictions...


TypeError: experiment2() missing 4 required positional arguments: 'impersonal_clf', 'impersonal_scaler', 'impersonal_features', and 'impersonal_labels'

In [79]:
results_df2.to_pickle(output_path+"experiment2.pickle")

In [80]:
results_df2.head()

,predictions,probabilities,true,user_id
0,"[Standing, Standing, Standing, Standing, Stand...","[[0.0275, 0.301, 0.1347, 0.4493, 0.0875], [0.0...","[Standing, Standing, Standing, Standing, Stand...",194
1,"[Sitting, Sitting, Sitting, Sitting, Sitting, ...","[[0.0069, 0.7652, 0.0576, 0.1623, 0.008], [0.0...","[Walking, Walking, Walking, Walking, Walking, ...",998
2,"[Standing, Standing, Sitting, Sitting, Sitting...","[[0.0395, 0.3601, 0.1746, 0.3622, 0.0636], [0....","[Standing, Standing, Standing, Standing, Stand...",1104
3,"[Standing, Standing, Standing, Standing, Stand...","[[0.0342, 0.3298, 0.1693, 0.418, 0.0487], [0.0...","[Standing, Standing, Standing, Standing, Stand...",1117
4,"[Standing, Sitting, Standing, Standing, Standi...","[[0.0416, 0.3661, 0.1653, 0.3813, 0.0457], [0....","[Standing, Standing, Standing, Standing, Stand...",1238


In [81]:
exp2_users = results_df2['user_id'].unique()

number_to_consider = 10

score_rows = []

for ind, user_row in results_df2.iterrows():
    confidence_ranking = np.argsort(np.max(user_row['probabilities'], axis=1))
    top30_predictions = user_row['predictions'][confidence_ranking[-number_to_consider:]]
    top30_true = user_row['true'][confidence_ranking[-number_to_consider:]]
    top30_score = accuracy_score(top30_true, top30_predictions)
    
    bottom30_predictions = user_row['predictions'][confidence_ranking[:number_to_consider]]
    bottom30_true = user_row['true'][confidence_ranking[:number_to_consider]]
    bottom30_score = accuracy_score(bottom30_true, bottom30_predictions)
    
    total_predictions = user_row['predictions']
    total_true = user_row['true']
    total_score = accuracy_score(total_true, total_predictions)
    
    score_row = {"user_id" : user_row['user_id'],
                 "top30" : top30_score,
                 "bottom30" : bottom30_score,
                 "total" : total_score}
    score_rows.append(score_row)
scores_df = pd.DataFrame(score_rows)

[165 167 166  47  50  40  37  44  51  39 159  49  41  43  67 164  53  38
  48  46 108  45 107 161 109  86 104 111  63  64 116  57  55 162  65 163
 110  60  42  62 101 114 113 160 146 143 155 153 133 141 100 145 156 137
 151 123 150 136 147 120 119  56 103 142 105  99 138 106 121 149 125 144
 148 117 115 130 127 102  54 124 152 112 132  66 129 118 158 128 126 154
 134 140  58  83 135 157 139  59 131  87  90  68 122  91  84  85  92  89
  77  95  69  98  79  97  73  94  75   6   9   7  76   8  80  10   5  72
  88  26  27  28  29  30   0  81  52  96  15  11  12  13  14  93  74  71
  70  82  78  23  24  25  21  22  61  17  18  19  20  16   3   4   1   2
  35  34  31  33  32  36]
[256 257 255 254 190 189 191 192 193 188 259 258 298 297  84 260  20  25
  26  21  22  23  24  29  28  27 261 113 109 115 114  77  79 155 156 157
 160  75 158 159 161  76  78  80  67  83  81 110 111 112  82  68  70  71
  66 116  48  69  54  52  74  73  72  49  56  55   0  51  50 183 184 108
 182 187   1 185 186  93 

In [82]:
confidence_ranking = np.argsort(np.max(user_row['probabilities'], axis=1))

In [83]:
confidence_ranking

array([33, 24, 41, 26, 25, 29, 35, 44, 40, 30, 42, 28, 39, 50, 45, 52, 48,
       49, 46, 43, 51, 47, 31, 32, 38, 27, 34, 36, 37, 53, 55, 58, 57, 59,
       60, 62, 61, 56, 54, 21, 69, 20, 65, 10, 11, 72, 68,  9,  8, 63, 12,
       13, 67, 70, 64, 23, 22,  1,  4,  6,  3,  5,  2, 66, 16, 17, 71,  0,
        7, 14, 15, 18, 19, 73])

In [84]:
user_row['user_id']

'1809'

In [85]:
np.max(user_row['probabilities'], axis=1)[confidence_ranking[-5:]]

array([ 0.5462,  0.5473,  0.5502,  0.5502,  0.555 ])

In [86]:
scores_df.head()

,bottom30,top30,total,user_id
0,0.4,1.0,0.702381,194
1,0.0,0.0,0.000000,998
2,0.0,1.0,0.541667,1104
3,0.0,0.9,0.598485,1117
4,0.0,0.7,0.351145,1238


In [87]:
trace = go.Bar(y=[scores_df['total'].mean(),
                  scores_df['top30'].mean(),
                  scores_df['bottom30'].mean()],
               x=['All Predictions', 'Top 30 Most Confident', 'Bottom 30 Least Confident'])


iplot([trace])

In [88]:
total_trace = go.Box(y=scores_df['total'],
                     name="All Predictions",
                     boxpoints="all",
                     jitter=0.3,
                     pointpos=-1.3)

top_trace = go.Box(y=scores_df['top30'],
                   name="Top 30 Most Confident Predictions",
                   boxpoints='all',
                   jitter=0.3,
                   pointpos=-1.3)

bottom_trace = go.Box(y=scores_df['bottom30'],
                      name="Bottom 30 Least Confident Predictions",
                      boxpoints='all',
                      jitter= 0.3,
                      pointpos=-1.3)

data = [total_trace, top_trace, bottom_trace]

layout = go.Layout(yaxis=dict(title="Accuracy"))

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Experiment 3

In [15]:
def experiment3(training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels):
    result_rows = []

    ignored_users = []
    print("predicting...")
    start_predicting = time.time()
    for user_id in wisdm.user_ids:
        print("User : %s" % user_id)
        user_df = wisdm.data_df[wisdm.data_df['user'] == user_id]

        if len(user_df) < (training_size + test_size):
            print("Not Enough Data, skipping...")
            ignored_users.append(user_id)
            continue

        personal_labels = np.array([t.decode("utf-8") for t in user_df['class'].as_matrix()])
        personal_features = user_df.as_matrix(columns=[user_df.columns[1:-1]])

        sss = StratifiedShuffleSplit(n_splits=4, test_size=test_size, train_size = training_size)

        personal_scores = []
        impersonal_scores = []
        hybrid_scores = []

        shuffle_count = 0
        try:
            for train_index, test_index in sss.split(personal_features, personal_labels):
                # data for personal model
                random_personal_features = personal_features[train_index]
                random_personal_labels = personal_labels[train_index]

                # create an active pool of everything not in the test set for active learning / hybrid model
                #active_pool_mask = np.ones(personal_labels.shape, dtype=bool)
                #active_pool_mask[test_index] = False
                #active_pool_features = personal_features[active_pool_mask]
                #active_pool_labels = personal_labels[active_pool_mask]

                # test set
                test_features = personal_features[test_index]
                test_labels = personal_labels[test_index]
                
                personal_counts = Counter(random_personal_labels)
                test_counts = Counter(test_labels)
                if shuffle_count == 0:
                    print("\tPersonal Training Counts : %s" % personal_counts)
                    print("\tPersonal Test Counts : %s" % test_counts)

                # build personal model and predict
                personal_predictions, _ = personal_pred(random_personal_features, random_personal_labels, test_features)
                personal_score = accuracy_score(test_labels, personal_predictions)
                personal_scores.append(personal_score)

                # build impersonal model and predict
                impersonal_predictions, impersonal_probabilities = impersonal_pred(impersonal_clf, test_features)
                impersonal_score = accuracy_score(test_labels, impersonal_predictions)
                impersonal_scores.append(impersonal_score)

                # build hybrid model and predict
                hybrid_predictions, _ = hybrid_pred(impersonal_features, impersonal_labels, \
                                                    random_personal_features, random_personal_labels, \
                                                    test_features)

                hybrid_score = accuracy_score(test_labels, hybrid_predictions)
                hybrid_scores.append(hybrid_score)
                #print("\t impersonal acc : %.3f" % impersonal_score)
                #print("\t personal acc : %.3f" % personal_score)
                #print("\t hybrid acc : %.3f" % hybrid_score)
                #print("")

                result_row = {"user_id" : user_id,
                              "shuffle" : shuffle_count,
                              "impersonal" : impersonal_score,
                              "personal" : personal_score,
                              "hybrid" : hybrid_score,
                              "training size" : training_size,
                              "test size" : test_size}
                result_rows.append(result_row)
                shuffle_count += 1
        except ValueError as ve:
            if "The least populated class" in ve.args[0]:
                print("\tNot enough labeled data for %s" % user_id)
                ignored_users.append(user_id)
                continue
            else:
                raise ve
        print("")
        print("\t impersonal acc : %.3f" % np.mean(impersonal_scores))
        print("\t personal acc : %.3f" % np.mean(personal_score))
        print("\t hybrid acc : %.3f" % np.mean(hybrid_score))
        print("")

    finished_predicting = time.time()
    print("Finished predicting in %s seconds" % (finished_predicting - start_predicting))
    print("Users without enough data : %s" % ignored_users)

    results_df = pd.DataFrame(result_rows)
    return results_df

In [17]:
training_sizes = [5,10,15,20]
test_size = 30
output_path = "./results/experiment3_10-5-17/"
results = []

# set impersonal model details
wisdm.set_data(version="1", make_compatible=True)
impersonal_df = wisdm.remove_all_nan(wisdm.data_df)
impersonal_labels = np.array([t.decode("utf-8") for t in impersonal_df['class'].as_matrix()])
impersonal_features = impersonal_df.as_matrix(columns=[impersonal_df.columns[1:-1]])
impersonal_scaler = StandardScaler().fit(impersonal_features)
scaled_train_X = impersonal_scaler.transform(impersonal_features)
impersonal_clf = wisdm.weka_RF()
impersonal_clf.set_params(n_estimators=n_trees, n_jobs=n_cores)

start=time.time()
print("Training Impersonal Model...")
impersonal_clf.fit(scaled_train_X, impersonal_labels)
finished_training = time.time()
print("Finished Training in %s seconds" % (finished_training - start))

wisdm.set_data(version="2", make_compatible=True)
for training_size in training_sizes:
    print("Personal Training Size : %s" % training_size)
    results_df = experiment3(training_size, test_size, impersonal_clf, impersonal_scaler, impersonal_features, impersonal_labels)
    results_df.to_pickle(output_path + "training_size-"+str(training_size)+".pickle")
    results.append(results_df)
    
results_df3 = pd.concat(results)

Training Impersonal Model...
Finished Training in 22.98333239555359 seconds
Personal Training Size : 5
predicting...
User : 194
	Personal Training Counts : Counter({'Standing': 2, 'Walking': 1, 'Jogging': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Standing': 12, 'Stairs': 7, 'Walking': 6, 'Jogging': 3, 'Sitting': 2})

	 impersonal acc : 0.700
	 personal acc : 0.900
	 hybrid acc : 0.900

User : 998
	Personal Training Counts : Counter({'Walking': 5})
	Personal Test Counts : Counter({'Walking': 30})

	 impersonal acc : 0.000
	 personal acc : 1.000
	 hybrid acc : 0.967

User : 1097
	Personal Training Counts : Counter({'Walking': 5})
	Personal Test Counts : Counter({'Walking': 30})

	 impersonal acc : 0.000
	 personal acc : 1.000
	 hybrid acc : 0.867

User : 1104
	Personal Training Counts : Counter({'Walking': 2, 'Standing': 1, 'Jogging': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Walking': 12, 'Standing': 8, 'Jogging': 4, 'Sitting': 3, 'Stairs': 3})

	 impersonal acc : 0.63


	 impersonal acc : 0.000
	 personal acc : 1.000
	 hybrid acc : 0.933

User : 1097
Not Enough Data, skipping...
User : 1104
	Personal Training Counts : Counter({'Walking': 4, 'Standing': 3, 'Sitting': 1, 'Jogging': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Walking': 12, 'Standing': 8, 'Jogging': 4, 'Sitting': 3, 'Stairs': 3})

	 impersonal acc : 0.575
	 personal acc : 0.767
	 hybrid acc : 0.800

User : 1117
	Personal Training Counts : Counter({'Standing': 6, 'Sitting': 3, 'Stairs': 1})
	Personal Test Counts : Counter({'Standing': 18, 'Sitting': 8, 'Stairs': 4})

	 impersonal acc : 0.617
	 personal acc : 0.900
	 hybrid acc : 0.900

User : 1205
Not Enough Data, skipping...
User : 1238
	Personal Training Counts : Counter({'Standing': 5, 'Walking': 3, 'Sitting': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Standing': 16, 'Walking': 10, 'Sitting': 2, 'Stairs': 2})

	 impersonal acc : 0.458
	 personal acc : 0.900
	 hybrid acc : 0.900

User : 1246
	Personal Training Counts : Co


	 impersonal acc : 0.592
	 personal acc : 0.833
	 hybrid acc : 0.967

User : 1205
Not Enough Data, skipping...
User : 1238
	Personal Training Counts : Counter({'Standing': 8, 'Walking': 5, 'Sitting': 1, 'Stairs': 1})
	Personal Test Counts : Counter({'Standing': 16, 'Walking': 10, 'Sitting': 2, 'Stairs': 2})

	 impersonal acc : 0.458
	 personal acc : 0.867
	 hybrid acc : 0.867

User : 1246
	Personal Training Counts : Counter({'Sitting': 15})
	Personal Test Counts : Counter({'Sitting': 30})

	 impersonal acc : 1.000
	 personal acc : 1.000
	 hybrid acc : 1.000

User : 1247
Not Enough Data, skipping...
User : 1253
	Personal Training Counts : Counter({'Sitting': 8, 'Walking': 7})
	Personal Test Counts : Counter({'Sitting': 17, 'Walking': 13})

	 impersonal acc : 0.567
	 personal acc : 1.000
	 hybrid acc : 1.000

User : 1269
Not Enough Data, skipping...
User : 1274
	Personal Training Counts : Counter({'Walking': 15})
	Personal Test Counts : Counter({'Walking': 30})

	 impersonal acc : 0.000


	 impersonal acc : 0.567
	 personal acc : 1.000
	 hybrid acc : 1.000

User : 1269
Not Enough Data, skipping...
User : 1274
	Personal Training Counts : Counter({'Walking': 20})
	Personal Test Counts : Counter({'Walking': 30})

	 impersonal acc : 0.000
	 personal acc : 1.000
	 hybrid acc : 1.000

User : 1276
Not Enough Data, skipping...
User : 1277
Not Enough Data, skipping...
User : 1280
Not Enough Data, skipping...
User : 1319
	Personal Training Counts : Counter({'Walking': 18, 'Sitting': 1, 'Jogging': 1})
	Personal Test Counts : Counter({'Walking': 27, 'Sitting': 1, 'Jogging': 1, 'Stairs': 1})

	 impersonal acc : 0.033
	 personal acc : 0.967
	 hybrid acc : 0.967

User : 1320
	Personal Training Counts : Counter({'Sitting': 13, 'Walking': 6, 'Stairs': 1})
	Personal Test Counts : Counter({'Sitting': 19, 'Walking': 9, 'Stairs': 2})

	 impersonal acc : 0.150
	 personal acc : 0.967
	 hybrid acc : 0.933

User : 1477
	Personal Training Counts : Counter({'Walking': 19, 'Standing': 1})
	Person